In [ ]:
print(os.listdir("/content/drive/MyDrive/"))


['Chapter 1.gslides', 'Colab Notebooks', 'processed_blur', 'processed_edge', 'Sharpened_Images', 'Blurred_Images', 'processed_images', 'face_classification_model.h5', 'classification_dataset', 'test.jpeg']


In [2]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, RMSprop, AdamW
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configurations
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
DATASET_PATH = "/content/drive/MyDrive/classification_dataset"
REQUIRED_CLASSES = ["side_profile", "criminal_dataset"]
EPOCHS = 10

def verify_dataset(path, required_classes):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Dataset path '{path}' does not exist!")

    available_classes = os.listdir(path)
    missing = [cls for cls in required_classes if cls not in available_classes]

    if missing:
        print(f"⚠ Warning: Missing classes: {missing}")
    print(f"✅ Dataset contains: {available_classes}")
    return available_classes

def create_generators(path, img_size, batch_size):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_gen = datagen.flow_from_directory(
        directory=path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='training'
    )
    val_gen = datagen.flow_from_directory(
        directory=path,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='binary',
        subset='validation'
    )
    return train_gen, val_gen

def build_model(input_shape=(128, 128, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),

        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def evaluate_model(optimizer_name, optimizer, train_gen, val_gen, epochs):
    print(f"\n🚀 Training with {optimizer_name} optimizer...\n")
    model = build_model()
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    history = model.fit(train_gen, validation_data=val_gen, epochs=epochs, verbose=1)

    precision = history.history.get("precision", [None])[-1]
    recall = history.history.get("recall", [None])[-1]
    f1_score = 2 * (precision * recall) / (precision + recall) if precision and recall and (precision + recall) != 0 else None

    return {
        "loss": history.history["loss"][-1],
        "val_loss": history.history["val_loss"][-1],
        "accuracy": history.history["accuracy"][-1],
        "val_accuracy": history.history["val_accuracy"][-1],
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score
    }

def main():
    verify_dataset(DATASET_PATH, REQUIRED_CLASSES)
    train_gen, val_gen = create_generators(DATASET_PATH, IMG_SIZE, BATCH_SIZE)
    print("📂 Class Indices:", train_gen.class_indices)

    optimizers = {
        'Adam': tf.keras.optimizers.Adam(),
        'SGD': SGD(),
        'RMSprop': RMSprop(),
        'AdamW': AdamW()
    }

    results = {}
    for name, opt in optimizers.items():
        results[name] = evaluate_model(name, opt, train_gen, val_gen, EPOCHS)

    results_df = pd.DataFrame(results).T
    print("\n📊 Final Results with F1-score:\n", results_df)

if __name__ == "__main__":
    main()


✅ Dataset contains: ['criminal_dataset', 'side_profile']
Found 3332 images belonging to 2 classes.
Found 831 images belonging to 2 classes.
📂 Class Indices: {'criminal_dataset': 0, 'side_profile': 1}

🚀 Training with Adam optimizer...

Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 36s 302ms/step - accuracy: 0.7356 - loss: 0.4875 - precision: 0.7354 - recall: 0.6244 - val_accuracy: 0.9723 - val_loss: 0.0823 - val_precision: 0.9589 - val_recall: 0.9851
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 29s 202ms/step - accuracy: 0.9754 - loss: 0.0750 - precision: 0.9778 - recall: 0.9709 - val_accuracy: 0.9844 - val_loss: 0.0497 - val_precision: 0.9779 - val_recall: 0.9901
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 203ms/step - accuracy: 0.9867 - loss: 0.0406 - precision: 0.9878 - recall: 0.9838 - val_accuracy: 0.9892 - val_loss: 0.0411 - val_precision: 0.9828 - val_recall: 0.9950
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 20s 193ms/step - accuracy: 0.9914 - loss: 0.0288 - precision: 0.9899 - recall: 0.9922